# Instalations

In [ ]:
!pip install openrouteservice folium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
! pip install polyline

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install geopy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Setting the api key

**Please acquire an api Key to be able to retrieve information from the openrouteservice API.**

In [ ]:
## Please set the obtained api key into this variable
key = 'Your api key'

# Importing Libraries

In [ ]:
import openrouteservice
from openrouteservice import convert
import folium
import json
import pandas as pd
import numpy as np

## Getting Routes using coordinates

In [ ]:
import requests
def get_directions_response(lat1, long1, lat2, long2):
  api_key = key
  client = openrouteservice.Client(key= api_key)
  coordinates = ((lat1, long1),(lat2, long2))
  response = client.directions(coordinates)

  response_route_summary = response['routes'][0]['summary']
  distance = float(response_route_summary['distance'])/1000 ## converting the distance into Km
  duration = float(response_route_summary['duration'])/60 ## converting the duration into Mins
   
  return response, client, coordinates, distance, duration

## Decoding the response

The response should be decoded into a usable data form to further be used in illustrating the map

In [ ]:
import polyline
def decode_geometry(client, coords):
    geometry = client.directions(coords)['routes'][0]['geometry']
    decoded = polyline.decode(geometry)
    
    return decoded

## Creating map

In [ ]:
import pandas as pd
import folium
def create_map(coordinates):
  ## a base form of map should be produced first
  m = folium.Map()

  print(coordinates[0], coordinates[-1])
  # add markers for the start and ending points on the base map
  for point in [coordinates[0], coordinates[-1]]:
      folium.Marker(point).add_to(m)

  # add the layer of lines to depict the route
  folium.PolyLine(coordinates, weight=5, opacity=1).add_to(m)

  # create optimal zoom to better show the locations and routes on the map
  m.fit_bounds([coordinates[0], coordinates[-1]])

  return m

# Testing the single location route extractor

In [ ]:
lat_1, long_1 = (8.34234,48.23424)
lat_2, long_2 = (8.34423,48.26424)
## given the locations latitude and longitude, the response, distance and duration will be produced
## the coordinations and response will then be used to illustrate the route produced by the service
response, client, coordinates, distance, duration = get_directions_response(lat_1, long_1, lat_2, long_2)

In [ ]:
print(f"{distance} Km")
print(f"{duration} Mins")

54.79 Km
13.396666666666667 Mins


In [ ]:
route_coordinates = decode_geometry(client, coordinates)

In [ ]:
coordinates_map = create_map(route_coordinates)

coordinates_map

(48.23383, 8.34427) (48.26355, 8.34343)


# Retrieving the Distance Matrix 

In [ ]:
def matrix_retreiving(coordinates):
  api_key = key
  client = openrouteservice.Client(key= api_key)
  matrix = client.distance_matrix(
      locations=coordinates,
      profile='foot-walking',
      metrics=['distance', 'duration'],
      validate=False,)
  return matrix

## Reading the Dataset

In [ ]:
def read_txt(file_path):
  df = pd.read_csv(file_path, delim_whitespace=True)
  return df

## Input

This section simply reads the file given.
**Plase modify the file path and set your choosen file.**

In [ ]:
file_path = "folder_path/Customers.txt"
data = read_txt(file_path)

In [ ]:
data.head()

,ID,LATITUDINE,LONGITUTIDE
0,256,43.30308,12.33330
1,261,43.04817,12.01046
2,393,43.02511,12.19077
3,423,43.11800,12.40036
4,424,43.18029,12.50907


In [ ]:
data.shape

(176, 3)

Creating a list of locations;

***Note: The openrouteservice accepts long/lat as input;***

In [ ]:
lattitudes = list(data['LATITUDINE'])
longitudes = list(data['LONGITUTIDE'])

locations = list()
for index in range(len(lattitudes)):
  locations.append([longitudes[index], lattitudes[index]])

In [ ]:
locations[0:20]

[[12.3333, 43.30308],
 [12.01046, 43.04817],
 [12.19077, 43.02511],
 [12.40036, 43.118],
 [12.50907, 43.18029],
 [12.0687, 43.02369],
 [12.46258, 43.16832],
 [12.59655, 43.32936],
 [12.39573, 43.08528],
 [12.09832, 43.02935],
 [12.33755, 43.30242],
 [12.5796, 43.34565],
 [12.38981, 43.09986],
 [12.24088, 43.06865],
 [12.4203, 42.77794],
 [12.48851, 42.84695],
 [12.13089, 43.0017],
 [12.34092, 42.98605],
 [12.28379, 42.98563],
 [12.32664, 43.30659]]

## Getting the matrix

A trivial test of how the function works, being operated on the first 25 items of the presented list of locations

In [ ]:
matrix = matrix_retreiving(locations[0:25])

In [ ]:
print(f"Durations in Seconds: {matrix['durations']}\n")
print(f"Distances in Meters: {matrix['distances']}")

Durations in Seconds: [[0.0, 41561.95, 32628.62, 22214.2, 20332.44, 38896.13, 16025.87, 23202.13, 24001.19, 36560.55, 318.93, 21698.36, 22759.45, 27034.02, 54705.65, 50630.29, 37837.68, 34790.07, 33889.09, 675.06, 26449.93, 23210.29, 83291.06, 75888.22, 71771.64], [41561.95, 0.0, 14980.61, 32023.26, 44064.96, 5367.68, 40623.61, 59846.05, 31911.45, 7094.34, 41684.78, 58342.28, 30844.09, 19035.59, 46857.97, 45045.29, 11512.6, 27779.04, 25177.93, 40924.11, 42739.88, 31270.92, 74703.02, 73873.82, 63923.96], [32628.62, 14980.61, 0.0, 20738.75, 32780.46, 10042.85, 29339.11, 49306.24, 18127.29, 7404.14, 32751.46, 47802.47, 17513.41, 6121.29, 32411.28, 30598.61, 5145.46, 13332.35, 10731.24, 31990.79, 29854.38, 18598.96, 60256.33, 53593.85, 49477.27], [22214.2, 32023.26, 20738.75, 0.0, 12847.2, 29357.44, 9450.64, 31686.47, 3416.73, 27021.86, 22175.69, 30182.69, 2174.98, 14620.37, 34121.19, 30045.82, 24199.9, 14205.61, 17419.32, 22819.4, 11522.12, 2625.83, 66500.73, 55303.76, 51187.18], [20332.4

# Main Function

You can simply run the code below;
First cell calculates the slice number by which the locations list is going to be sliced as there is a length limit of how many locations can be computed by the openrouteservice API.

Therefore, the main locations list is divided into lists of 50 length that are also divided into two different lists of 25 items which will be altered in the main loop according to the indices of the locations list in order to calculate the whole distance matrix considering each possible pair of locations. Thus, a matrix of 50*50 elements will be produced at each step, except the two occasions in which the length of list is less than 50. For instance, for the presented list, the remaining items is 1 which shows the remaining locations that couldn't be considered in the previous steps. in this regard, this circumstances are different in shape than the typical considered shape and they are set into the main distance matrix in a different way.
The code below should clarify what is being done.

Note: there is a **500 number per day** limit on the amount of operating matrix distance calculation, imposed by the openrouteservice API. If this limit is exceeded, it is stated that there will be no operating resources available until the following day.

In [ ]:
shift_count = 25
slice_number = int(len(locations) / shift_count)
remaining_count = len(locations) % shift_count
print(slice_number, remaining_count)
print("--------------")

7 1
--------------


In [ ]:
distance_matrix = np.zeros(shape=(data.shape[0], data.shape[0]))
duration_matrix = np.zeros(shape=(data.shape[0], data.shape[0]))

begining_index1 = 0
ending_index1 = shift_count

begining_index2 = 0
ending_index2 = shift_count
cntr1 = -1
cntr2 = -1

while cntr1 <= slice_number-1:
  cntr1 += 1
  cntr2 = -1

  begining_index1 = cntr1*shift_count
  if cntr1 == slice_number:
    ending_index1 = begining_index1 + remaining_count
  else:
    ending_index1 = begining_index1 + shift_count
  print(begining_index1, ending_index1)
  while cntr2 <= slice_number-1:
      cntr2 += 1
      print(f"cntr_1 {cntr1} and cntr_2 {cntr2}")
      begining_index2 = cntr2*shift_count
      if cntr2 == slice_number:
        ending_index2 = begining_index2 + remaining_count
      else:
        ending_index2 = begining_index2 + shift_count

      print("first =>",begining_index1, ending_index1,"second =>", begining_index2, ending_index2)

      location_coordinates = []
      location_coordinates.extend(locations[begining_index1: ending_index1])
      location_coordinates.extend(locations[begining_index2: ending_index2])

      print(len(location_coordinates))
      matrix = matrix_retreiving(location_coordinates)
      matrices_distance = np.reshape(matrix['distances'], (-1, len(matrix['distances'][0])))
      matrices_duration = np.reshape(matrix['durations'], (-1, len(matrix['durations'][0])))
      mat_shape = matrices_distance.shape
      print(f"shape of matrix {mat_shape}")
      if cntr1 == slice_number and cntr2 == slice_number:
        index = int(remaining_count/2)+1
        print("first", index)
        distance_matrix[begining_index1:ending_index1, begining_index1:ending_index1] = matrices_distance [0:index, 0:index]
        distance_matrix[begining_index1:ending_index1, begining_index2:ending_index2] = matrices_distance [0:index, index:]
        distance_matrix[begining_index2:ending_index2, begining_index1:ending_index1] = matrices_distance [index:, 0:index]
        distance_matrix[begining_index2:ending_index2, begining_index2:ending_index2] = matrices_distance [index:, index:]

        duration_matrix[begining_index1:ending_index1, begining_index1:ending_index1] = matrices_duration [0:index, 0:index]
        duration_matrix[begining_index1:ending_index1, begining_index2:ending_index2] = matrices_duration [0:index, index:]
        duration_matrix[begining_index2:ending_index2, begining_index1:ending_index1] = matrices_duration [index:, 0:index]
        duration_matrix[begining_index2:ending_index2, begining_index2:ending_index2] = matrices_duration [index:, index:]
      
      elif cntr2 == slice_number:
        print("first")
        distance_matrix[begining_index1:ending_index1, begining_index1:ending_index1] = matrices_distance [0:25, 0:25]
        distance_matrix[begining_index1:ending_index1, begining_index2:ending_index2] = matrices_distance [0:25, 25:25+remaining_count]
        distance_matrix[begining_index2:ending_index2, begining_index1:ending_index1] = matrices_distance [25:25+remaining_count, 0:25]
        distance_matrix[begining_index2:ending_index2, begining_index2:ending_index2] = matrices_distance [25:25+remaining_count, 25:25+remaining_count]

        duration_matrix[begining_index1:ending_index1, begining_index1:ending_index1] = matrices_duration [0:25, 0:25]
        duration_matrix[begining_index1:ending_index1, begining_index2:ending_index2] = matrices_duration [0:25, 25:25+remaining_count]
        duration_matrix[begining_index2:ending_index2, begining_index1:ending_index1] = matrices_duration [25:25+remaining_count, 0:25]
        duration_matrix[begining_index2:ending_index2, begining_index2:ending_index2] = matrices_duration [25:25+remaining_count, 25:25+remaining_count]
      
      elif cntr1 == slice_number:
        print("third")
        distance_matrix[begining_index1:ending_index1, begining_index1:ending_index1] = matrices_distance [0:remaining_count, 0:remaining_count]
        distance_matrix[begining_index1:ending_index1, begining_index2:ending_index2] = matrices_distance [0:remaining_count, remaining_count:]
        distance_matrix[begining_index2:ending_index2, begining_index1:ending_index1] = matrices_distance [remaining_count:, 0:remaining_count]
        distance_matrix[begining_index2:ending_index2, begining_index2:ending_index2] = matrices_distance [remaining_count:, remaining_count:]
      
        duration_matrix[begining_index1:ending_index1, begining_index1:ending_index1] = matrices_duration [0:remaining_count, 0:remaining_count]
        duration_matrix[begining_index1:ending_index1, begining_index2:ending_index2] = matrices_duration [0:remaining_count, remaining_count:]
        duration_matrix[begining_index2:ending_index2, begining_index1:ending_index1] = matrices_duration [remaining_count:, 0:remaining_count]
        duration_matrix[begining_index2:ending_index2, begining_index2:ending_index2] = matrices_duration [remaining_count:, remaining_count:]
      
      else:
        print("forth")
        distance_matrix[begining_index1:ending_index1, begining_index1:ending_index1] = matrices_distance [0:25, 0:25]
        distance_matrix[begining_index1:ending_index1, begining_index2:ending_index2] = matrices_distance [0:25, 25:50]
        distance_matrix[begining_index2:ending_index2, begining_index1:ending_index1] = matrices_distance [25:50, 0:25]
        distance_matrix[begining_index2:ending_index2, begining_index2:ending_index2] = matrices_distance [25:50, 25:50]
        
        duration_matrix[begining_index1:ending_index1, begining_index1:ending_index1] = matrices_duration [0:25, 0:25]
        duration_matrix[begining_index1:ending_index1, begining_index2:ending_index2] = matrices_duration [0:25, 25:50]
        duration_matrix[begining_index2:ending_index2, begining_index1:ending_index1] = matrices_duration [25:50, 0:25]
        duration_matrix[begining_index2:ending_index2, begining_index2:ending_index2] = matrices_duration [25:50, 25:50]

        
        
      print("================")
  print("=-=-=-=-=-=-=-=-=-=-=-=-=-=-=")

0 25
cntr_1 0 and cntr_2 0
first => 0 25 second => 0 25
50
shape of matrix (50, 50)
forth
cntr_1 0 and cntr_2 1
first => 0 25 second => 25 50
50
shape of matrix (50, 50)
forth
cntr_1 0 and cntr_2 2
first => 0 25 second => 50 75
50
shape of matrix (50, 50)
forth
cntr_1 0 and cntr_2 3
first => 0 25 second => 75 100
50
shape of matrix (50, 50)
forth
cntr_1 0 and cntr_2 4
first => 0 25 second => 100 125
50
shape of matrix (50, 50)
forth
cntr_1 0 and cntr_2 5
first => 0 25 second => 125 150
50
shape of matrix (50, 50)
forth
cntr_1 0 and cntr_2 6
first => 0 25 second => 150 175
50
shape of matrix (50, 50)
forth
cntr_1 0 and cntr_2 7
first => 0 25 second => 175 176
26
shape of matrix (26, 26)
first
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
25 50
cntr_1 1 and cntr_2 0
first => 25 50 second => 0 25
50
shape of matrix (50, 50)
forth
cntr_1 1 and cntr_2 1
first => 25 50 second => 25 50
50
shape of matrix (50, 50)
forth
cntr_1 1 and cntr_2 2
first => 25 50 second => 50 75
50
shape of matrix (50, 50)
forth
cnt

# Ilustrating the results in Dataframe format

In [ ]:
df_distance = pd.DataFrame(distance_matrix)
print(df_distance.shape)
df_distance

(176, 176)


,0,1,2,3,4,5,6,7,8,9,...,166,167,168,169,170,171,172,173,174,175
0,0.00,57725.09,45317.66,30853.18,28239.57,54022.55,22258.21,32225.25,33335.15,50778.68,...,40302.85,30416.66,61915.38,28412.38,111807.30,25675.00,45738.65,61791.88,46443.52,43070.84
1,57725.09,0.00,20806.47,44476.95,61201.66,7455.13,56421.95,83119.76,44321.59,9853.28,...,63189.94,81311.17,79554.95,27014.64,99512.13,51000.27,92137.05,50892.18,44129.15,81712.68
2,45317.66,20806.47,0.00,28804.00,45528.70,13948.46,40749.00,68481.12,25176.88,10283.57,...,45524.97,66672.53,59873.50,17958.90,79447.23,35327.32,76464.10,30827.29,24064.26,80566.45
3,30853.18,44476.95,28804.00,0.00,17843.44,40774.41,13125.97,44009.13,4745.54,37530.54,...,19831.72,42200.54,40023.45,24167.88,88487.32,7642.06,48778.84,33202.26,17853.91,69372.09
4,28239.57,61201.66,45528.70,17843.44,0.00,57499.12,6204.11,27247.81,21089.69,54255.24,...,17126.39,25944.34,40730.18,36649.91,95559.41,12222.84,31667.03,45815.24,30554.91,71213.39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,25675.00,51000.27,35327.32,7642.06,12222.84,47297.73,8133.10,37201.10,10888.31,44053.86,...,15469.18,35897.63,36986.49,30457.81,86878.41,0.00,43158.24,36149.14,20888.80,68648.82
172,45738.65,92137.05,76464.10,48778.84,31667.03,88434.52,37553.26,14789.82,48194.18,85190.64,...,34546.30,16227.47,50758.93,67999.06,110636.14,43158.24,0.00,72142.43,57522.07,79006.86
173,61791.88,50892.18,30827.29,33202.26,45815.24,45049.71,43185.91,73211.35,28501.81,40369.28,...,38832.43,71907.88,41709.90,47316.80,49563.37,36149.14,72142.43,0.00,16829.91,100083.01
174,46443.52,44129.15,24064.26,17853.91,30554.91,38286.68,27925.58,57951.02,13153.46,33606.25,...,25358.82,56647.55,40241.75,30871.36,65462.71,20888.80,57522.07,16829.91,0.00,85116.53


In [ ]:
df_duration = pd.DataFrame(duration_matrix)
print(df_duration.shape)
df_duration

(176, 176)


,0,1,2,3,4,5,6,7,8,9,...,166,167,168,169,170,171,172,173,174,175
0,0.00,41561.95,32628.62,22214.20,20332.44,38896.13,16025.87,23202.13,24001.19,36560.55,...,29017.95,21899.94,44578.91,20456.86,80501.05,18485.95,32931.71,44489.95,33439.18,31010.80
1,41561.95,0.00,14980.61,32023.26,44064.96,5367.68,40623.61,59846.05,31911.45,7094.34,...,45496.52,58543.86,57279.36,19450.47,71648.48,36720.00,66338.43,36642.27,31772.89,58832.95
2,32628.62,14980.61,0.00,20738.75,32780.46,10042.85,29339.11,49306.24,18127.29,7404.14,...,32777.85,48004.05,43108.75,12930.36,57201.80,25435.49,55053.92,22195.59,17326.20,58007.61
3,22214.20,32023.26,20738.75,0.00,12847.20,29357.44,9450.64,31686.47,3416.73,27021.86,...,14278.76,30384.28,28816.76,17400.75,63710.71,5502.24,35120.67,23905.49,12854.72,49947.65
4,20332.44,44064.96,32780.46,12847.20,0.00,41399.14,4466.94,19618.38,15184.46,39063.56,...,12330.95,18679.87,29325.62,26387.83,68802.59,8800.41,22800.24,32986.82,21999.42,51273.39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,18485.95,36720.00,25435.49,5502.24,8800.41,34054.18,5855.79,26784.71,7839.50,31718.60,...,11137.76,25846.20,26630.16,21929.47,62552.30,0.00,31073.87,26027.26,15039.86,49426.90
172,32931.71,66338.43,55053.92,35120.67,22800.24,63672.61,27038.31,10648.61,34699.69,61337.02,...,24873.28,11683.70,36546.32,48959.20,79657.77,31073.87,0.00,51942.38,41415.76,56884.66
173,44489.95,36642.27,22195.59,23905.49,32986.82,32435.70,31093.71,52711.99,20521.22,29065.80,...,27959.23,51773.48,30031.02,34067.95,35685.48,26027.26,51942.38,0.00,12117.48,72059.40
174,33439.18,31772.89,17326.20,12854.72,21999.42,27566.32,20106.31,41724.59,9470.45,24196.42,...,18258.28,40786.07,28973.94,22227.29,47132.98,15039.86,41415.76,12117.48,0.00,61283.63


# Saving the Matrices (Distance and Duration)

In [ ]:
## saving the results in the csv format
## The directory should be modified to the desired folder path 
import os  
os.makedirs('/content/drive/MyDrive/Routing_results/', exist_ok=True)  
df_distance.to_csv('/content/drive/MyDrive/Routing_results/Distance_Matrix.csv') 

df_duration.to_csv('/content/drive/MyDrive/Routing_results/Duration_Matrix.csv') 

In [ ]:
## saving the results in the text format

np.savetxt(r'/content/drive/MyDrive/Routing_results/Distance_Matrix.txt', df_distance.values, fmt='%d')

np.savetxt(r'/content/drive/MyDrive/Routing_results/Duration_Matrix.txt', df_duration.values, fmt='%d')